In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.externals import joblib
import warnings
warnings.filterwarnings('ignore')

### Model

In [3]:
def GS_CV(alg, xtrain, ytrain,parameters, njobs):
    alg_gs = GridSearchCV(estimator=alg, param_grid = parameters, scoring='accuracy', n_jobs = njobs, cv=5, iid = False, refit = True)
    alg_gs.fit(xtrain,ytrain)
    return alg_gs
def get_accuracy(alg, X_train, X_test, y_train64, y_test64, y_train10, data):
    alg_ = alg.best_estimator_.fit(X_train, y_train10)
    joblib.dump(alg_, 'model_rf_%s.pkl'%data)
    train_probs = alg_.predict_proba(X_train)
    test_probs = alg_.predict_proba(X_test)
    c_train = np.argmax(train_probs, axis=1)
    c_test = np.argmax(test_probs, axis=1)
    train_acc = sum(c_train == y_train64)/len(y_train64)
    test_acc = sum(c_test == y_test64)/len(y_test64)
    
    print("Train Accuracy: %.4f" %(train_acc))
    print("Test Accuracy: %.4f" %(test_acc))
    return None

In [4]:
def GAP_vector(pred, conf, true, return_x=False):
    x = pd.DataFrame({'pred': pred, 'conf': conf, 'true': true})
    x.sort_values('conf', ascending=False, inplace=True, na_position='last')
    x['correct'] = (x.true == x.pred).astype(int)
    x['prec_k'] = x.correct.cumsum() / (np.arange(len(x)) + 1)
    x['term'] = x.prec_k * x.correct
    gap = x.term.sum() / x.true.count()
    if return_x:
        return gap, x
    else:
        return gap

### Training (Original)

In [6]:
#READ NPY FILE:
X_train64 = np.load('X_train64.npy')
X_test64 = np.load('X_test64.npy')

y_train64 = []
y_test64 = []
#READ TXT FILE:
with open("y_train64.txt", "r") as f:
     for line in f:
            y_train64.append(int(line.strip()))
            
with open("y_test64.txt", "r") as f:
     for line in f:
            y_test64.append(int(line.strip()))

print(X_train64.shape)
print(X_test64.shape)

(17510, 12288)
(11674, 12288)


In [7]:
y_train10 = label_binarize(y_train64, classes=[i for i in range(10)])
y_test10 = label_binarize(y_test64, classes=[i for i in range(10)])
hy_paras = {'estimator__n_estimators':[20,50,100],
            'estimator__criterion':['gini','entropy'], 'estimator__min_samples_split':[2,10,50,250]}

In [7]:
rf = RandomForestClassifier(random_state = 20, verbose = 1)
rf_ovr = OneVsRestClassifier(rf)
rf_gs =  GS_CV(rf_ovr, X_train64, y_train10, hy_paras, 2)

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   15.8s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   15.8s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   24.9s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   25.4s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   26.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   26.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   29.5s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   30.3s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   22.2s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   22.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   30.4s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   29.8s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   21.2s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   21.8s finished
[Parallel(n_jobs=1)]

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  1.4min finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   39.6s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   57.7s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   59.9s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   59.7s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  1.1min finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   49.2s finished
[Parallel(n_jobs=1)]

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.6min finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  3.0min finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  2.4min finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  2.4min finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  2.4min finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  3.0min finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  2.8min finished
[Parallel(n_jobs=1)]

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   25.3s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   15.5s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   23.4s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   29.8s finished
[Parallel(n_jobs=1)]

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   59.4s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.2min finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  1.2min finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   56.0s finished
[Parallel(n_jobs=1)]

...

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   18.2s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   17.8s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   43.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   12.6s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.1min finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.3min finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.2min finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.1min finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.4min finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.4min finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.2min finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.2min finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.0min finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    7.9s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    9.4s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    9.1s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   10.3s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.1min finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   10.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   10.3s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    8.9s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    9.5s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    8.6s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    7.6s finished
[Parallel(n_jobs=1)]

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   24.4s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   19.8s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   25.6s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   23.6s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   24.9s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   23.5s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   25.6s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   25.8s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   22.9s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   24.4s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   24.0s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   25.8s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   21.9s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   22.9s finished
[Parallel(n_jobs=1)]

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   54.5s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   42.5s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   49.0s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   49.5s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   49.6s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   49.0s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   46.3s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   39.0s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]

In [9]:
rf_gs.best_estimator_

OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=20, verbose=1, warm_start=False),
          n_jobs=1)

In [20]:
get_accuracy(rf_gs, X_train64, X_test64, y_train64, y_test64, y_train10, "original")

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   16.6s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   18.5s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   19.4s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   21.1s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   18.8s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   24.7s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   20.3s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   19.2s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   17.5s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   14.8s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]

Train Accuracy: 0.9992
Test Accuracy: 0.5057


[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.1s finished


In [3]:
rf_org = joblib.load('model_rf_original.pkl')

In [9]:
train_probs = rf_org.predict_proba(X_train64)
test_probs = rf_org.predict_proba(X_test64)
c_train = np.argmax(train_probs, axis=1)
c_test = np.argmax(test_probs, axis=1)

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]

In [23]:
conf_test = list(map(lambda x: max(x), test_probs))

In [26]:
GAP_vector(c_test, conf_test, y_test64, return_x=False)

0.39526389329809225

### Training (Filtered)

In [5]:
#READ NPY FILE:
X_train64 = np.load('X_train_filter64.npy')
X_test64 = np.load('X_test_filter64.npy')

y_train64 = []
y_test64 = []
#READ TXT FILE:
with open("y_train_filter64.txt", "r") as f:
     for line in f:
            y_train64.append(int(line.strip()))
            
with open("y_test_filter64.txt", "r") as f:
     for line in f:
            y_test64.append(int(line.strip()))

print(X_train64.shape)
print(X_test64.shape)

(13273, 12288)
(8850, 12288)


In [7]:
y_train10 = label_binarize(y_train64, classes=[i for i in range(10)])
y_test10 = label_binarize(y_test64, classes=[i for i in range(10)])
hy_paras = {'estimator__n_estimators':[15,25,50],
            'estimator__criterion':['gini','entropy'], 'estimator__min_samples_split':[2,10,25,50]}

In [10]:
rf = RandomForestClassifier(random_state = 20, verbose = 1)
rf_ovr = OneVsRestClassifier(rf)
rf_gs =  GS_CV(rf_ovr, X_train64, y_train10, hy_paras, 2)

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    9.0s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    9.4s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   14.3s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   16.0s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   17.9s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   16.0s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   20.3s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   20.9s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   15.9s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   16.8s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   21.4s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   21.3s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   13.5s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   13.4s finished
[Parallel(n_jobs=1)]

[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   31.1s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   16.2s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   24.4s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   25.7s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   34.3s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   28.0s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   24.4s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   32.1s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   25.1s finished
[Parallel(n_jobs=1)]

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   36.7s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  1.0min finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   54.9s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  1.2min finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  1.0min finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   51.0s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  1.2min finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  1.3min finished
[Parallel(n_jobs=1)]

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   15.9s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    9.9s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   16.1s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   21.1s finished
[Parallel(n_jobs=1)]

[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   28.3s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   33.0s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   33.0s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   25.6s finished
[Parallel(n_jobs=1)]

...

[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   26.7s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   17.7s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   15.3s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   27.2s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   13.5s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   55.6s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  1.1min finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   58.3s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   55.7s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   59.4s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  1.0min finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   52.5s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   53.3s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  1.0min finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   47.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   15.4s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   49.6s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]

[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   27.7s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   26.0s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   31.7s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   25.3s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   25.9s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   26.8s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   28.3s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   27.0s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   24.8s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   22.4s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   24.4s finished
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   59.7s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   49.4s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   51.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   50.8s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   43.3s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]

In [11]:
rf_gs.best_estimator_

OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=15, n_jobs=1,
            oob_score=False, random_state=20, verbose=1, warm_start=False),
          n_jobs=1)

In [12]:
get_accuracy(rf_gs, X_train64, X_test64, y_train64, y_test64, y_train10, "filtered")

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   10.5s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   10.3s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   10.7s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    9.7s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    8.7s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   10.5s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    9.2s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   12.0s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    9.3s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    8.4s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]

Train Accuracy: 0.9990
Test Accuracy: 0.5080


[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.0s finished


In [13]:
rf_org = joblib.load('model_rf_filtered.pkl')

In [15]:
train_probs = rf_org.predict_proba(X_train64)
test_probs = rf_org.predict_proba(X_test64)
c_train = np.argmax(train_probs, axis=1)
c_test = np.argmax(test_probs, axis=1)

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]

In [16]:
conf_test = list(map(lambda x: max(x), test_probs))

In [17]:
GAP_vector(c_test, conf_test, y_test64, return_x=False)

0.39705159423159542